In [10]:
from __future__ import unicode_literals
import classifiers
import re
import nltk
from Politweet import get_tweets, get_transcript
import ratings
from sentiment import polarity_train, classify, prob_classify, plus_df, minus_df
import pandas as pd
pd.set_option('display.max_colwidth', 1200)

In [ ]:
tweets = get_tweets("./datasets/tweets.tsv")
transcript = get_transcript("./datasets/transcript.csv")
todo = set(tweets.index)

In [3]:
tweets.to_pickle('./datasets/tweets.pickle')

In [4]:
polarity = polarity_train(tweets)
polarity_minus = set(i for i,_ in classify(minus_df(tweets), polarity))
polarity_plus = set(i for i,_ in classify(plus_df(tweets), polarity))
polarities = tweets.reindex(polarity_minus | polarity_plus).sort()
print len(polarities), "- polarity"
todo -= set(polarities.index)

482 - polarity


/Library/Python/2.7/site-packages/pandas/core/strings.py:189: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  " groups, use str.extract.", UserWarning)


In [5]:
others = ratings.all(tweets, ratings.OTHER)
print len(others), "- marked as OTHERS"
todo -= set(others.index)

151 - marked as OTHERS


In [6]:
print len(todo), "- untagged"

2605 - untagged


In [7]:
left = tweets.reindex(todo).sort()
left[:1]

,pub.date.GMT,content,author.name,author.nickname,rating.1,rating.2,rating.3,rating.4,rating.5,rating.6,rating.7,rating.8,clean,tokens
tweet.id,,,,,,,,,,,,,,
936466790,9/27/08 1:00,Jim Lehrer just directed the debate audience ... 30 seconds ... #tweetdebate,B_E_X,Becky L,4,2,4,NaN,NaN,NaN,NaN,NaN,"[{u'lemma': u'Jim', u'token': u'Jim', u'pos': u'NNP'}, {u'lemma': u'Lehrer', u'token': u'Lehrer', u'pos': u'NNP'}, {u'lemma': u'direct', u'token': u'directed', u'pos': u'VBD'}, {u'lemma': u'debate', u'token': u'debate', u'pos': u'NN'}, {u'lemma': u'audience', u'token': u'audience', u'pos': u'NN'}, {u'lemma': u'...', u'token': u'...', u'pos': u':'}, {u'lemma': u'30', u'token': u'30', u'pos': u'CD'}, {u'lemma': u'second', u'token': u'seconds', u'pos': u'NNS'}, {u'lemma': u'...', u'token': u'...', u'pos': u':'}, {u'lemma': u'#tweetdebate', u'token': u'#tweetdebate', u'pos': u'NN'}]","[Jim, Lehrer, direct, debate, audience, 30, second, #tweetdebate]"


In [8]:
polarities[["content","tokens", "clean"]]

,content,tokens,clean
tweet.id,,,
936475544,Obama -2 it an occupation not war #tweetdebate,"[Obama, -2, occupation, war, #tweetdebate]","[{u'lemma': u'Obama', u'token': u'Obama', u'pos': u'NNP'}, {u'lemma': u'-2', u'token': u'-2', u'pos': u'-NONE-'}, {u'lemma': u'occupation', u'token': u'occupation', u'pos': u'NN'}, {u'lemma': u'war', u'token': u'war', u'pos': u'NN'}, {u'lemma': u'#tweetdebate', u'token': u'#tweetdebate', u'pos': u'JJ'}]"
936475791,Obama +1 for swift and wise #tweetdebate,"[Obama, +1, swift, wise, #tweetdebate]","[{u'lemma': u'Obama', u'token': u'Obama', u'pos': u'NNP'}, {u'lemma': u'+1', u'token': u'+1', u'pos': u'NNP'}, {u'lemma': u'swift', u'token': u'swift', u'pos': u'NN'}, {u'lemma': u'wise', u'token': u'wise', u'pos': u'VB'}, {u'lemma': u'#tweetdebate', u'token': u'#tweetdebate', u'pos': u'NN'}]"
936475857,"+1 for Obama ""Moving fast, moving swiftly"" #tweetdebate","[+1, Obama, Moving, fast, move, swiftly, #tweetdebate]","[{u'lemma': u'+1', u'token': u'+1', u'pos': u'NN'}, {u'lemma': u'Obama', u'token': u'Obama', u'pos': u'NNP'}, {u'lemma': u'``', u'token': u'``', u'pos': u'``'}, {u'lemma': u'Moving', u'token': u'Moving', u'pos': u'NNP'}, {u'lemma': u'fast', u'token': u'fast', u'pos': u'RB'}, {u'lemma': u',', u'token': u',', u'pos': u','}, {u'lemma': u'move', u'token': u'moving', u'pos': u'VBG'}, {u'lemma': u'swiftly', u'token': u'swiftly', u'pos': u'RB'}, {u'lemma': u'''', u'token': u'''', u'pos': u''''}, {u'lemma': u'#tweetdebate', u'token': u'#tweetdebate', u'pos': u'NN'}]"
936477055,+1 for Obama having 4 clear objectives to his plan #tweetdebate,"[+1, Obama, 4, clear, objective, plan, #tweetdebate]","[{u'lemma': u'+1', u'token': u'+1', u'pos': u'NN'}, {u'lemma': u'Obama', u'token': u'Obama', u'pos': u'NNP'}, {u'lemma': u'4', u'token': u'4', u'pos': u'CD'}, {u'lemma': u'clear', u'token': u'clear', u'pos': u'JJ'}, {u'lemma': u'objective', u'token': u'objectives', u'pos': u'NNS'}, {u'lemma': u'plan', u'token': u'plan', u'pos': u'NN'}, {u'lemma': u'#tweetdebate', u'token': u'#tweetdebate', u'pos': u'JJ'}]"
936477336,""" Obama +1 for articulating his plans - initial & recognition #tweetdebate""","[Obama, +1, articulating, plan, initial, &, recognition, #tweetdebate]","[{u'lemma': u'``', u'token': u'``', u'pos': u'``'}, {u'lemma': u'Obama', u'token': u'Obama', u'pos': u'NNP'}, {u'lemma': u'+1', u'token': u'+1', u'pos': u'-NONE-'}, {u'lemma': u'articulating', u'token': u'articulating', u'pos': u'NN'}, {u'lemma': u'plan', u'token': u'plans', u'pos': u'NNS'}, {u'lemma': u'-', u'token': u'-', u'pos': u':'}, {u'lemma': u'initial', u'token': u'initial', u'pos': u'JJ'}, {u'lemma': u'&', u'token': u'&', u'pos': u'CC'}, {u'lemma': u'recognition', u'token': u'recognition', u'pos': u'NN'}, {u'lemma': u'#tweetdebate', u'token': u'#tweetdebate', u'pos': u'NN'}, {u'lemma': u'''', u'token': u'''', u'pos': u''''}]"
936478235,Obama +1 on taxpayer protection and fair shake #tweetdebate,"[Obama, +1, taxpayer, protection, fair, shake, #tweetdebate]","[{u'lemma': u'Obama', u'token': u'Obama', u'pos': u'NNP'}, {u'lemma': u'+1', u'token': u'+1', u'pos': u'NNP'}, {u'lemma': u'taxpayer', u'token': u'taxpayer', u'pos': u'NN'}, {u'lemma': u'protection', u'token': u'protection', u'pos': u'NN'}, {u'lemma': u'fair', u'token': u'fair', u'pos': u'JJ'}, {u'lemma': u'shake', u'token': u'shake', u'pos': u'NN'}, {u'lemma': u'#tweetdebate', u'token': u'#tweetdebate', u'pos': u'NN'}]"
936478277,Obama +1 #tweetdebate,"[Obama, +1, #tweetdebate]","[{u'lemma': u'Obama', u'token': u'Obama', u'pos': u'NNP'}, {u'lemma': u'+1', u'token': u'+1', u'pos': u'NNP'}, {u'lemma': u'#tweetdebate', u'token': u'#tweetdebate', u'pos': u'NN'}]"
936478329,#tweetdebate +3 to Obama - the failed economy is because of the last 8 years of Bush and McCain's support for those policies,"[#tweetdebate, +3, Obama, fail, economy, last, 8, year, Bush, McCain, 's, support, policy]","[{u'lemma': u'#tweetdebate', u'token': u'#tweetdebate', u'pos': u'NN'}, {u'lemma': u'+3', u'toke

In [23]:
for i, tweet in tweets[:3].iterrows():
    print [token["lemma"] for token in tweet["clean"]]

[u'Watching', u'#tweetdebate', u'drink', u':', u'(', u'wait', u'start', u'cringing', u'McCain', u'blunder']
[u'@ahg3', u'@MichDot', u'Yeah', u',', u'slime', u'actually', u'second', u'choice', u',', u'ca', u"n't", u'say', u'first', u'one', u'.', u'Okay', u',', u"'re", u'roll', u'...']
[u'Preparing', u'heart', u'attack', u'#tweetdebate']
